In [1]:
import os
import argparse
import socket
import time

import tensorboard_logger as tb_logger
import torch
import torch.optim as optim
import torch.nn as nn
import torch.backends.cudnn as cudnn


from models import model_dict
from models.util import Embed, ConvReg, LinearEmbed
from models.util import Connector, Translator, Paraphraser

from dataset.cifar100 import get_cifar100_dataloaders, get_cifar100_dataloaders_sample

from helper.util import adjust_learning_rate

from distiller_zoo import DistillKL, HintLoss, Attention, Similarity, Correlation, VIDLoss, RKDLoss
from distiller_zoo import PKT, ABLoss, FactorTransfer, KDSVD, FSP, NSTLoss
from crd.criterion import CRDLoss
from ntk import NTKLoss

from helper.loops import train_distill as train, validate
from helper.pretrain import init

import numpy as np

# Utilities

In [2]:
# returns the number of parameters in the model or a part of the model
def num_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum([np.prod(p.size()) for p in model_parameters]) 

In [10]:
# Load the teacher
resnet50_rp = model_dict['resnet50_rp'](num_classes=100)
resnet50 = model_dict['ResNet50'](num_classes=100)

model_path = "save/models/ResNet50_vanilla/ckpt_epoch_240.pth"

resnet50.load_state_dict(torch.load(model_path)['model'])

<All keys matched successfully>

In [6]:
num_parameters(resnet50_rp)

23705252

In [9]:
num_parameters(resnet50)

23705252

In [11]:
resnet50_rp.load_state_dict(torch.load(model_path)['model'])

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.conv.weight", "conv1.bn.weight", "conv1.bn.bias", "conv1.bn.running_mean", "conv1.bn.running_var", "layer1.0.conv1.conv.weight", "layer1.0.conv1.bn.weight", "layer1.0.conv1.bn.bias", "layer1.0.conv1.bn.running_mean", "layer1.0.conv1.bn.running_var", "layer1.0.conv2.conv.weight", "layer1.0.conv2.bn.weight", "layer1.0.conv2.bn.bias", "layer1.0.conv2.bn.running_mean", "layer1.0.conv2.bn.running_var", "layer1.0.conv3.conv.weight", "layer1.0.conv3.bn.weight", "layer1.0.conv3.bn.bias", "layer1.0.conv3.bn.running_mean", "layer1.0.conv3.bn.running_var", "layer1.0.downsample.conv.weight", "layer1.0.downsample.bn.weight", "layer1.0.downsample.bn.bias", "layer1.0.downsample.bn.running_mean", "layer1.0.downsample.bn.running_var", "layer1.1.conv1.conv.weight", "layer1.1.conv1.bn.weight", "layer1.1.conv1.bn.bias", "layer1.1.conv1.bn.running_mean", "layer1.1.conv1.bn.running_var", "layer1.1.conv2.conv.weigh

In [13]:
resnet50_rp

ResNet(
  (conv1): ConvBlock(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=False, momentum=0.1, affine=True, track_running_stats=True)
  )
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): ConvBlock(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=False, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2): ConvBlock(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=False, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv3): ConvBlock(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=False, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
      (downsample): ConvBlock(
        (conv

In [14]:
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (